In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [4]:
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

Trying to download new driver from http://chromedriver.storage.googleapis.com/80.0.3987.106/chromedriver_win32.zip
Unpack archive C:\Users\Madik\.wdm\drivers\chromedriver\80.0.3987.106\win32\chromedriver.zip


In [6]:
# Extract description of a post from Instagram link
driver.get('https://www.instagram.com/p/B8gu8PZBDSY/')
soup = BeautifulSoup(driver.page_source,"lxml")
desc = " "
 
for item in soup.findAll('a'):
    desc= desc + " " + str(item.string)
 
# Extract tag list from Instagram post description
taglist = desc.split()
taglist = [x for x in taglist if x.startswith('#')]
index = 0
while index < len(taglist):
    taglist[index] = taglist[index].strip('#')
    index += 1
 
print(taglist)

['dancer', 'dance', 'music', 'dancelove', 'dancing', 'dancers', 'dancevideo', 'dancepage', 'challenge', 'explore', 'choreography', 'hiphop', 'dancehiphop', 'dancelife', 'danceforlife', 'joshkillacky', 'mattsteffanina', 'seanandkaycee', 'trending', 'dytto', 'baileysok', 'worldofdance', '18yearsold', 'dancesport', 'immabeast', 'millenium', 'joshbeauchamp', 'gabedeguzman', 'jadebug98']


In [7]:
# Define dataframe to store hashtag information
tag_df  = pd.DataFrame(columns = ['Hashtag', 'Number of Posts','Posting Freq (mins)'])
 
# Loop over each hashtag to extract information
for tag in taglist:
     
    driver.get('https://www.instagram.com/explore/tags/'+str(tag))
    soup = BeautifulSoup(driver.page_source,"lxml")
     
    # Extract current hashtag name
    tagname = tag
    # Extract total number of posts in this hashtag
    # NOTE: Class name may change in the website code
    # Get the latest class name by inspecting web code
    try:
        nposts = soup.find('span', {'class': 'g47SY'}).get_text()
    except AttributeError:
        print("ok")
         
    # Extract all post links from 'explore tags' page
    # Needed to extract post frequency of recent posts
    myli = []
    for a in soup.find_all('a', href=True):
        myli.append(a['href'])
 
    # Keep link of only 1st and 9th most recent post 
    newmyli = [x for x in myli if x.startswith('/p/')]
    del newmyli[:9]
    del newmyli[9:]
    del newmyli[1:8]
    
    timediff = []
 
    # Extract the posting time of 1st and 9th most recent post for a tag
    for j in range(len(newmyli)):
        driver.get('https://www.instagram.com'+str(newmyli[j]))
        soup = BeautifulSoup(driver.page_source,"lxml")
 
        for i in soup.findAll('time'):
            if i.has_attr('datetime'):
                timediff.append(i['datetime'])
                #print(timediff)
                
        # Calculate time difference between posts
        # For obtaining posting frequency
        datetimeFormat = '%Y-%m-%dT%H:%M:%S.%fZ'
        date1 = timediff[1]
        date2 = timediff[0]
        diff = datetime.datetime.strptime(date1, datetimeFormat)\
        - datetime.datetime.strptime(date2, datetimeFormat)
        pfreq= int(diff.total_seconds()/(9*60))

     
    # Add hashtag info to dataframe
    tag_df.loc[len(tag_df)] = [tagname, nposts,pfreq]
    
# Check the final dataframe
print(tag_df)
 
# CSV output for hashtag analysis
tag_df.to_csv('hashtag_list.csv')

           Hashtag Number of Posts Posting Freq (mins)
0           dancer      27 627 882                   0
1            dance      96 356 854                   0
2            music     298 577 445                   0
3        dancelove         660 348                   0
4          dancing      18 742 417                   0
5          dancers      11 412 330                   0
6       dancevideo       1 266 957                   0
7        dancepage          18 909                   0
8        challenge      16 448 075                   0
9          explore     112 793 690                   0
10    choreography       4 889 600                   0
11          hiphop      78 063 307                   0
12     dancehiphop          31 444                   0
13       dancelife       5 404 056                   0
14    danceforlife         343 322                   0
15    joshkillacky           9 780                   0
16  mattsteffanina          71 825                   0
17   seana